<a href="https://colab.research.google.com/github/Eugene2705/MITx_Supply_Chain_Optimisation/blob/main/SCM_275x_Purchasing_Costs_and_Generalized_Capacities_Python_Exercise_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Purchasing Costs and Generalized Capacities - Python Exercise**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [ ]:
# Install necessary packages if they are not already installed

!pip install gurobipy   # Gurobi optimization solver
!pip install pandas     # Pandas for data analysis and manipulation
!pip install folium     # Folium for creating interactive maps
!pip install geopy      # Geopy for computing distances and working with geographic data


In [ ]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


## **Helper functions**

### **Ploting nodes on a map**

In [ ]:


# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [ ]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi')

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Ploting flows of different products on a map**

In [ ]:
# Defining a function to plot flows on a map using folium

def plot_flows_products(map,          # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               product_colors,        # Dictionary representing colors of the lines representing flows of different products
               max_width = 30,        # Maximum line width for the flows, default is 30
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key, product), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key, product), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

          # Get the coordinates of the nodes for plotting the line
          points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


          # Add a PolyLine to the map to represent the flow between the nodes
          folium.PolyLine(points,
                            color=product_colors[product],              # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map

## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [ ]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/purchasing_costs_gen_capacities/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,lat,lon
0,c1,The Hague,52.0800,4.3100
1,c2,Aveiro,40.6333,-8.6500
2,c3,Geneva,46.2017,6.1469
3,c4,Bydgoszcz,53.1219,18.0003
4,c5,Milan,45.4669,9.1900


In [ ]:
# File containing demand data
demand_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/purchasing_costs_gen_capacities/demand_data.csv'

# Loading demand data file into a pandas DataFrame
demand_data_df = pd.read_csv(demand_data_file)

# Displaying the first few rows of the DataFrame to verify the data
demand_data_df.head()

,customerID,product,demand
0,c1,product_a,275
1,c1,product_b,165
2,c1,product_c,220
3,c2,product_a,0
4,c2,product_b,100


In [ ]:
# File containing supplier data
supplier_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/purchasing_costs_gen_capacities/suppliers.csv'

# Loading supplier data into a pandas DataFrame
suppliers_df = pd.read_csv(supplier_data_file)

# Displaying the first few rows of the DataFrame to verify the data
suppliers_df.head()

,ID,name,lat,lon
0,s1,Bassano del Grappa,45.7667,11.7342
1,s2,Torrejon de Ardoz,40.4614,-3.4978
2,s3,Merida,38.9000,-6.3333
3,s4,Susice,49.2312,13.5202
4,s5,Veendam,53.1000,6.8833


In [ ]:
# File containing supply data
supply_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/purchasing_costs_gen_capacities/supply_data.csv'

# Loading supply data into a pandas DataFrame
supply_data_df = pd.read_csv(supply_data_file)

# Displaying the first few rows of the DataFrame to verify the data
supply_data_df.head()

,supplierID,product,supply,var_cost
0,s1,product_a,0,0
1,s1,product_b,0,0
2,s1,product_c,6000,28
3,s2,product_a,5740,28
4,s2,product_b,2880,24


In [ ]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/purchasing_costs_gen_capacities/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,lat,lon,capacity,fixed_cost
0,w1,Fiorenzuola d'Arda,44.9333,9.9000,20000,10000
1,w2,Hille,52.3331,8.7500,20000,10000
2,w3,Waltershausen,50.8975,10.5558,20000,10000
3,w4,Levallois-Perret,48.8950,2.2872,20000,10000
4,w5,Cambrai,50.1767,3.2356,20000,10000


In [ ]:
# Reading file containing warehousing data
warehousing_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/purchasing_costs_gen_capacities/warehousing_data.csv'

# Loading warehousing data into a pandas DataFrame
warehousing_data_df = pd.read_csv(warehousing_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehousing_data_df.head()

,warehouseID,product,resources,var_cost
0,w1,product_a,1.2,4.6
1,w1,product_b,2.0,4.2
2,w1,product_c,1.3,5.0
3,w2,product_a,1.2,4.7
4,w2,product_b,2.0,5.0


#### Definition of Classes

In [ ]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon):
        self.ID = ID              # Customer's ID
        self.name = name          # Customer's name
        self.lat = lat            # Customer's latitude
        self.lon = lon            # Customer's longitude

        self.demand = dict()      # Initializing an empty dictionnary to store customers's demand per product

        self.active = True        # Initializing node as active


In [ ]:
# Class representing a Supplier object

class Supplier():
    def __init__(self, ID, name, lat, lon):
        self.ID = ID                    # Supplier's ID
        self.name = name                # Supplier's name
        self.lat = lat                  # Supplier's latitude
        self.lon = lon                  # Supplier's longitude

        self.supply = dict()            # Initializing an empty dictionnary to store supplier's available supply per product
        self.var_cost = dict()          # Initializing an empty dictionnary to store supplier's variable cost (i.e., purchase cost) per product

        self.active = True              # Initializing node as active

In [ ]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, fixed_cost, capacity):
        self.ID = ID                  # Warehouse's ID
        self.name = name              # Warehouse's name
        self.lat = lat                # Warehouse's latitude
        self.lon = lon                # Warehouse's longitude
        self.fixed_cost = fixed_cost  # Warehouse's fixed cost
        self.capacity = capacity      # Warehouse's capacity

        self.resources = dict()       # Initializing an empty dictionnary to store warehouse's required ressources for product
        self.var_cost = dict()        # Initializing an empty dictionnary to store warehouse's variable costs for product

        self.active = True            # Initializing node as active


#### Creating node objects

In [ ]:
# Initializing an empty dictionary to store node objects
nodes = dict()

In [ ]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'])         # Customer's longitude

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [ ]:
customers['c1'].demand

{}

In [ ]:
demand_data_df.head()

,customerID,product,demand
0,c1,product_a,275
1,c1,product_b,165
2,c1,product_c,220
3,c2,product_a,0
4,c2,product_b,100


In [ ]:
for i, row in demand_data_df.iterrows():
  # print(row['customerID'], row['product'], row['demand'])
  # print(customers[row['customerID']])
  # print(customers[row['customerID']].demand)
  customers[row['customerID']].demand[row['product']] = row['demand']
  # print(customers[row['customerID']].demand)
  # break

In [ ]:
customers['c1'].demand

customers['c5'].demand

{'product_a': 274, 'product_b': 548, 'product_c': 411}

In [ ]:
# Associating demand data to customers
for i, row in demand_data_df.iterrows():
    customers[row['customerID']].demand[row['product']] = row['demand']

In [ ]:
# Visualizing an example of customer demand
customers['c1'].demand

{'product_a': 275, 'product_b': 165, 'product_c': 220}

In [ ]:
# Creating a dictionary of supplier objects
suppliers = dict()
for i, row in suppliers_df.iterrows():
    suppliers[row['ID']] = Supplier(ID=row['ID'],           # Supplier's ID
                                    name=row['name'],       # Supplier's name
                                    lat=row['lat'],         # Supplier's latitude
                                    lon=row['lon'])         # Supplier's longitude

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**customers, **suppliers}

In [ ]:
# Associating supply data to suppliers
for i, row in supply_data_df.iterrows():
    suppliers[row['supplierID']].supply[row['product']] = row['supply']
    suppliers[row['supplierID']].var_cost[row['product']] = row['var_cost']

In [ ]:
# Visualizing an example of supplier's available supply
suppliers['s1'].supply

{'product_a': 0, 'product_b': 0, 'product_c': 6000}

In [ ]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],               # Warehouse's ID
                                    name = row['name'],             # Warehouse's name
                                    lat = row['lat'],               # Warehouse's latitude
                                    lon = row['lon'],               # Warehouse's longitude
                                    fixed_cost = row['fixed_cost'], # Warehouse's fixed cost
                                    capacity = row['capacity'])     # Warehouse's fixed cost

# Merging the suppliers dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

In [ ]:
# Associating warehousing data to warehouses
for i, row in warehousing_data_df.iterrows():
    warehouses[row['warehouseID']].resources[row['product']] = row['resources']
    warehouses[row['warehouseID']].var_cost[row['product']] = row['var_cost']


In [ ]:
# Visualizing an example of warehouse's ressources
warehouses['w1'].resources

{'product_a': 1.2, 'product_b': 2.0, 'product_c': 1.3}

#### Visualizing node objects

In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([50, 10.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [ ]:
# Creating a dictionary containing distances between suppliers and warehouses and warehouses and customers

distances = dict()
for s, supplier in suppliers.items():
  for w, warehouse in warehouses.items():
      distances[s, w] = compute_geodesic_distance(origin = supplier, destination = warehouse, unit = 'km')

for w, warehouse in warehouses.items():
  for c, customer in customers.items():
      distances[w, c] = compute_geodesic_distance(origin = warehouse, destination = customer, unit = 'km')

#### Arc costs

In [ ]:
cost_unit_km_supplier_warehouse = 0.2 # Cost per unit per kilometer from supplier to warehouse
cost_unit_km_warehouse_customer = 0.3 # Cost per unit per kilometer from warehouse to customer

# Creating a dictionary containing unit costs between suppliers and warehouses, and between warehouses and customers
unit_cost = dict()

for s, supplier in suppliers.items():                                                          # Iterate over suppliers
    for w, warehouse in warehouses.items():                                                    # Iterate over warehouses
        unit_cost[s, w] = distances[s, w] * cost_unit_km_supplier_warehouse                    # Calculate unit cost as distance multiplied by cost per km (supplier to warehouse)

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        unit_cost[w, c] = distances[w, c] * cost_unit_km_warehouse_customer                    # Calculate unit cost as distance multiplied by cost per km (warehouse to customer)


### **Products**

In [ ]:
# Defining a list of products to consider in the model
products = ['product_a', 'product_b', 'product_c']

## **Optimization model**

### **Creating and solving the optimization model**

***❗Task 1: modify the following code to account for the fact that the warehouses w4 and w5 cannot handle 'product_a'***

In [ ]:
# Initializing the optimization model for purchasing costs and generalized capacities
model = grb.Model("Purchasing Costs and Generalized Capacities")

# Creating binary decision variables for warehouse location selection
wh_location_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over warehouses
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                               name = "wh_location_vars_{0}".format(w))

# Creating continuous decision variables for flows
flow_vars = dict()
for s, supplier in suppliers.items():  # Iterate over suppliers
    for w, warehouse in warehouses.items():  # Iterate over warehouses
        for p in products:  # Iterate over products
            flow_vars[s, w, p] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                                   name = "flow_vars_{0}_{1}_{2}".format(s, w, p))

for w, warehouse in warehouses.items():  # Iterate over warehouses
    for c, customer in customers.items():  # Iterate over customers
        for p in products:  # Iterate over products
            flow_vars[w, c, p] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_vars_{0}_{1}_{2}".format(w, c, p))

# Defining the objective function (minimizing total cost)

# Fixed cost for opening warehouses
fixed_cost_warehouses = grb.quicksum(wh_location_vars[w]
                          * warehouse.fixed_cost
                          for w, warehouse in warehouses.items())


# Variable cost of suppliers
var_cost_supplier = grb.quicksum(flow_vars[s, w, p]
                        * supplier.var_cost[p]
                        for s, supplier in suppliers.items()
                        for w, warehouse in warehouses.items()
                        for p in products)

# Variable cost of warehouses
var_cost_warehouse = grb.quicksum(flow_vars[w, c, p]
                        * warehouse.var_cost[p]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Transportation cost from suppliers to warehouses
cost_supplier_warehouse = grb.quicksum(unit_cost[s, w] * flow_vars[s, w, p]
                        for s, supplier in suppliers.items()
                        for w, warehouse in warehouses.items()
                        for p in products)

# Transportation cost from warehouses to customers
cost_warehouse_customer = grb.quicksum(unit_cost[w, c] * flow_vars[w, c, p]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Total cost is the sum of all cost components (fixed costs, variable costs, transportation costs)
total_cost = fixed_cost_warehouses + var_cost_supplier + var_cost_warehouse + cost_supplier_warehouse + cost_warehouse_customer

# Setting the objective to minimize total cost
model.setObjective(total_cost, grb.GRB.MINIMIZE)

# Adding demand constraints (the flow to each customer must meet their demand)
for c, customer in customers.items():
    for p in products:
        model.addConstr(grb.quicksum(flow_vars[w, c, p] for w, warehouse in warehouses.items()) == customer.demand[p])

# Adding supply constraints (the flow from each supplier must not exceed their supply)
for s, supplier in suppliers.items():
    for p in products:
        model.addConstr(grb.quicksum(flow_vars[s, w, p] for w, warehouse in warehouses.items()) <= supplier.supply[p])

# Adding flow preservation constraints (the flow into a warehouse must equal the flow out)
for w, warehouse in warehouses.items():
    for p in products:
        model.addConstr(grb.quicksum(flow_vars[s, w, p] for s, supplier in suppliers.items()) ==
                        grb.quicksum(flow_vars[w, c, p] for c, customer in customers.items()))

# Adding warehouse capacity constraints (the ressources consumed by the flow through a warehouse must not exceed its capacity)
for w, warehouse in warehouses.items():
    model.addConstr(grb.quicksum(flow_vars[s, w, p] * warehouse.resources[p] for s, supplier in suppliers.items() for p in products) <= wh_location_vars[w] * warehouse.capacity)


# model.addConstr(grb.quicksum(flow_vars[s, 'w5', 'product_a'] for s, supplier in suppliers.items()) == 0)
# model.addConstr(grb.quicksum(flow_vars[s, 'w4', 'product_a'] for s, supplier in suppliers.items()) == 0)


# for w in ['w4', 'w5']:
#   for p in ['product_a']:
#     model.addConstr(grb.quicksum(flow_vars[s, w, p] for s, supplier in suppliers.items()) == 0)


for s, supplier in suppliers.items():
  model.addConstr(flow_vars[s, 'w5', 'product_a'] == 0)
  model.addConstr(flow_vars[s, 'w4', 'product_a'] == 0)



# Solving the model
model.optimize()

# Updating the status of warehouses based on the optimization results
for w, warehouse in warehouses.items():
  warehouse.active = True if wh_location_vars[w].X == 1 else False

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 164 rows, 1090 columns and 2426 nonzeros
Model fingerprint: 0x0ffbc9ae
Variable types: 1080 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 6e+03]
Presolve removed 53 rows and 404 columns
Presolve time: 0.01s
Presolved: 111 rows, 686 columns, 1502 nonzeros
Variable types: 676 continuous, 10 integer (10 binary)
Found heuristic solution: objective 1814276.0083

Root relaxation: objective 1.731419e+06, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  

## **Solution visualization and analysis**

In [ ]:
print('Fixed Cost Warehouses: ', fixed_cost_warehouses.getValue())

print('Variable Cost Suppliers: ', var_cost_supplier.getValue())
print('Variable Cost Warehouses: ', var_cost_warehouse.getValue())

print('Transportation Cost Warehouse to Customer: ', cost_warehouse_customer.getValue())
print('Transportation Cost Supplier to Warehouse: ', cost_supplier_warehouse.getValue())


Fixed Cost Warehouses:  60000.0
Variable Cost Suppliers:  252093.0
Variable Cost Warehouses:  36125.39999999999
Transportation Cost Warehouse to Customer:  906327.5830299074
Transportation Cost Supplier to Warehouse:  535812.6093917119


In [ ]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([50, 10.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and white background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='white')

# Plot supplier locations with an industry icon, orange color, and yellow background
plot_nodes(map=map, nodes=suppliers, icon='industry', active_color='orange', background_color='yellow')

# Define different colors for plotting products
colors = {'product_a': 'red',
          'product_b': 'green',
          'product_c': 'blue'}

# Plot flows on the map
plot_flows_products(map = map, vars = flow_vars, nodes = nodes, product_colors = colors, opacity = 0.3)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


In [ ]:
customers['c6'].demand

{'product_a': 0, 'product_b': 200, 'product_c': 0}